In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from urllib.parse import urlparse, urlunparse
from webdriver_manager.chrome import ChromeDriverManager

def extract_business_links_and_names():
    business_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/Toolkit/Jobs/find-businesses-detail.aspx?')]")
    for link in business_links:
        business_finder_url = link.get_attribute('href')
        business_name = link.text
        business_finder_urls.append(business_finder_url)
        businessNames.append(business_name)
industry_names = [
    "110000",
    "210000",
    "220000",
    "230000",
    "310000",
    "320000",
    "330000",
    "420000",
    "440000",
    "450000",
    "480000",
    "490000",
    "510000",
    "520000",
    "530000",
    "540000",
    "550000",
    "560000",
    "610000",
    "620000",
    "710000",
    "720000",
    "810000",
    "920000"
]
page_limits = [
    1154, 1080, 199, 10672, 1551
]
options = webdriver.ChromeOptions()
#options.add_argument("--headless")
#options.add_argument("--disable-gpu")
#options.add_argument("--no-sandbox")
#options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
curPage = 1
for industry_name in industry_names:
    business_finder_urls = []
    businessNames = []
    while True:
        url = "https://www.careeronestop.org/Toolkit/Jobs/find-businesses-results.aspx?curPage="+str(curPage)+"&keyword="+industry_name+"&location=California&lang=en&pagesize=50"
        driver.get(url)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/Toolkit/Jobs/find-businesses-detail.aspx?')]")))
            extract_business_links_and_names()
            curPage += 1
        except:
            curPage = 1
            break
    data = {
    "Business Name": businessNames,
    "Business Finder Website": [site for site in business_finder_urls],
    "Industry": industry_name
    }
    df = pd.DataFrame(data)
    df.to_excel(industry_name + ".xlsx", index=False)
driver.quit()

In [ ]:
list_files = [
    "11 - Agriculture, Forestry, Fishing and Hunting",
    "21 - Mining, Quarrying, and Oil and Gas Extraction",
    "22 - Utilities",
    "23 - Construction",
    "31-33 - Manufacturing",
    "42 - Wholesale Trade",
    "44-45 - Retail Trade",
    "48-49 - Transportation and Warehousing",
    "51 - Information",
    "52 - Finance and Insurance",
    "53 - Real Estate and Rental and Leasing",
    "54 - Professional, Scientific, and Technical Services",
    "55 - Management of Companies and Enterprises",
    "56 - Administrative and Support and Waste Management and Remediation Services",
    "61 - Educational Services",
    "62 - Health Care and Social Assistance",
    "71 - Arts, Entertainment, and Recreation",
    "72 - Accommodation and Food Services",
    "81 - Other Services (except Public Administration)",
    "92 - Public Administration"
]
initial = pd.DataFrame()
for file in list_files:
    df = pd.read_excel("Raw Data/" + file + ".xlsx")
    initial = pd.concat([initial, df], ignore_index = True)
initial = initial.drop_duplicates()
initial.head()

In [ ]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from urllib.parse import urlparse, urlunparse
from webdriver_manager.chrome import ChromeDriverManager
def extract_details(driver, url):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cos-table-detail'))
        )
        table = driver.find_element(By.CLASS_NAME, 'cos-table-detail')
        industry_code = None
        website = None
        rows = table.find_elements(By.TAG_NAME, 'tr')
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, 'td')
            if len(cells) > 1:
                if 'Industry Code' in cells[0].text:
                    industry_code = cells[1].text.strip()[:2]  # Get first 2 digits
                elif 'Website' in cells[0].text:
                    website = cells[1].find_element(By.TAG_NAME, 'a').get_attribute('href')
        return industry_code, website
    except Exception as e:
        print(f"An error occurred with {url}: {e}")
    return None, None
final = initial.copy()
industry_names = ["11", "21", "22", "23", "31", "32", "33", "42", "44", "45", "48", "49", "51", "52", "53", "54", "55", "56", "61", "62", "71", "72", "81", "92"]
industry_dict = {code: 0 for code in industry_names}
options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
industry_codes = []
business_urls = []
names = []
final["Sampled"] = False
count = 0
while count < 20 * 150:
    random_index = np.random.choice(final.index, size=1, replace=False)[0]
    if final["Sampled"][random_index] == False:
        final.at[random_index, "Sampled"]= True
        business_finder_url = final['Business Finder Website'][random_index]
        industry_code, website = extract_details(driver, business_finder_url)
        if website is not None:
            count += 1
            names.append(final['Business Name'][random_index])
            industry_codes.append(industry_code)
            business_urls.append(website)
            #industry_dict[industry_code] += 1
driver.quit()
sample = pd.DataFrame({'Name': names, 'Industry Code': industry_codes, 'Business Homepage': business_urls})
sample.to_excel("accurate_random_sample_2.xlsx", index=False)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from urllib.parse import urlparse, urlunparse
from webdriver_manager.chrome import ChromeDriverManager
from boilerpy3 import extractors
from urllib.request import Request, urlopen
import re
import string

def get_final_url(url, headers):
    try:
        response = requests.head(url, headers=headers, allow_redirects=True)
        return response.url
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return url
    
def f2(g):
        priorities = ["privacy policy", "privacy", "policy", "privacy center", "privacy statement", "legal",
                      "legal notice", "privacy notice", "data protection", "data privacy",
                      "privacy and terms", "security and privacy", "privacy & security"
                      "confidentiality", "gdpr compliance", "privacy practices"]
        for key in priorities:
            if key in g:
                return g.get(key)
        return None

def get_privacy_policy_link(homepage_url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    homepage_url = get_final_url(homepage_url, headers)
    try:
        response = requests.get(homepage_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if response.status_code == 406:
            print(f"406 Not Acceptable error for {homepage_url}")
            return "Error fetching homepage", homepage_url
        elif response.status_code == 403:
            print(f"403 Forbidden error for {homepage_url}")
            return "Error fetching homepage", homepage_url
        else:
            print(f"HTTP error for {homepage_url}: {e}")
        return "Error fetching homepage", homepage_url
    except requests.RequestException as e:
        print(f"Error fetching {homepage_url}: {e}")
        return "Error fetching homepage", homepage_url
    soup = BeautifulSoup(response.content, 'html.parser')
    dict = {}
    for link in soup.find_all('a', href=True):
        if 'privacy' in link.text.lower() or 'policy' in link.text.lower() or 'legal' in link.text.lower():
            curr = link['href']
            full_url = requests.compat.urljoin(homepage_url, curr)
            if curr != 'https://policies.google.com/privacy' and full_url != homepage_url:
                dict.update({link.text.lower():full_url})
    return f2(dict), homepage_url

def extract_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    except requests.exceptions.HTTPError as e:
        if response.status_code == 406:
            print(f"406 Not Acceptable error for {url}")
        elif response.status_code == 403:
            print(f"403 Forbidden error for {url}")
            return handle_forbidden_error(url)
        else:
            print(f"HTTP error for {url}: {e}")
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
    return ""

def handle_forbidden_error(url):
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.text
    except requests.exceptions.HTTPError as e:
        print(f"Still getting HTTP error for {url}: {e}")
    except requests.RequestException as e:
        print(f"Error fetching {url} with modified user-agent: {e}")
    return ""

def check_mentions(url, keywords):
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        html = urlopen(req).read()
        soup = BeautifulSoup(html, features="html.parser")
        for script in soup(["script", "style"]):
            script.extract()    # rip it out
        text = soup.get_text()
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)
        ans = []
        text = text.lower()
        keywords = [k.lower() for k in keywords]
        for k in keywords:
            if k in text:
                indeces = [m.start() for m in re.finditer(k, text)]
                for i in indeces:
                    if text[i - 1] != '\n' and text[i+len(k)] != '\n':
                        ans.append(k.capitalize())
                        break
        return ans
    except:
        return []

df = pd.read_excel("accurate_random_sample_2.xlsx") # accurate_random_sample_2
homepages = df['Business Homepage']
privacy_policy_links = []
gdpr_mentioned = []
colorado_mentioned = []
other_states_mentioned = []

for i, homepage_url in enumerate(homepages):
    privacy_policy_link, final_homepage_url = get_privacy_policy_link(homepage_url)
    df.at[i, 'Business Homepage'] = final_homepage_url
    if privacy_policy_link == "Error fetching homepage":
        privacy_policy_links.append("Error fetching homepage")
        gdpr_mentioned.append("Error fetching homepage")
        colorado_mentioned.append("Error fetching homepage")
        other_states_mentioned.append("Error fetching homepage")
    elif privacy_policy_link:
        mentions = check_mentions(privacy_policy_link, ['Gdpr', 'General data protection regulation', 'Colorado', "Alabama", "Alaska", "American Samoa", "Arizona", "Arkansas", "Connecticut", "Delaware", "District of Columbia", "Florida", "Georgia", "Guam", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Minor Outlying Islands", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Northern Mariana Islands", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "U.S. Virgin Islands", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"])
        if mentions != "Boilerplate error":
            privacy_policy_links.append(privacy_policy_link)
            gdpr_mentioned.append('Gdpr' in mentions or 'General data protection regulation' in mentions)
            colorado_mentioned.append('Colorado' in mentions)
            to_remove = ['Gdpr', 'General data protection regulation', 'Colorado']
            for m in list(mentions):
                if m in to_remove:
                    mentions.remove(m)
            other_states_mentioned.append(', '.join(mentions))
        else:
            privacy_policy_links.append(privacy_policy_link)
            gdpr_mentioned.append('Error fetching privacy policy page')
            colorado_mentioned.append('Error fetching privacy policy page')
            other_states_mentioned.append('Error fetching privacy policy page')
    else:
        privacy_policy_links.append("Homepage does not link to privacy page")
        gdpr_mentioned.append('')
        colorado_mentioned.append('')
        other_states_mentioned.append('')

df['Privacy Policy Link'] = privacy_policy_links
df['GDPR Mentioned'] = gdpr_mentioned
df['Colorado Mentioned'] = colorado_mentioned
df['Other States Mentioned'] = other_states_mentioned
df.to_excel("accurate_random_sample_2_1.xlsx", index=False)

In [ ]:
import pandas as pd
df = pd.read_excel("accurate_random_sample_2_1.xlsx")
df = df.dropna(subset=["Privacy Policy Link"])
df = df.drop_duplicates(subset=["Business Homepage"], keep=False)
df = df[df['Privacy Policy Link'] != 'Error fetching homepage']
df = df[df['GDPR Mentioned'] != 'Error fetching privacy policy page']
#df["GDPR Mentioned"] = df["GDPR Mentioned"].fillna(False)
#df["Colorado Mentioned"] = df["Colorado Mentioned"].fillna(False)
df.to_excel("Cleaned Data v3.xlsx", index=False)

In [ ]:
def f2(g):
        priorities = ["privacy policy", "privacy", "policy", "privacy center", "privacy statement", "legal",
                      "legal notice", "privacy notice", "data protection", "data privacy",
                      "privacy and terms", "security and privacy",
                      "confidentiality", "gdpr compliance", "privacy practices"]
        for key in priorities:
            if key in g:
                return g.get(key)
        return None
def get_privacy_policy_link_mobile(homepage_url):
    headers = {"User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15A372 Safari/604.1"}
    homepage_url = get_final_url(homepage_url, headers)
    try:
        response = requests.get(homepage_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if response.status_code == 406:
            print(f"406 Not Acceptable error for {homepage_url}")
            return "Error fetching homepage", homepage_url
        elif response.status_code == 403:
            print(f"403 Forbidden error for {homepage_url}")
            return "Error fetching homepage", homepage_url
        else:
            print(f"HTTP error for {homepage_url}: {e}")
        return "Error fetching homepage", homepage_url
    except requests.RequestException as e:
        print(f"Error fetching {homepage_url}: {e}")
        return "Error fetching homepage", homepage_url
    soup = BeautifulSoup(response.content, 'html.parser')
    dict = {}
    for link in soup.find_all('a', href=True):
        if 'privacy' in link.text.lower() or 'policy' in link.text.lower() or 'legal' in link.text.lower():
            curr = link['href']
            full_url = requests.compat.urljoin(homepage_url, curr)
            if curr != 'https://policies.google.com/privacy' and full_url != homepage_url:
                dict.update({link.text.lower():full_url})
    return f2(dict)
df = pd.read_excel("Cleaned Data v4.xlsx")
mobile_policy_links = []
mobile_policy_links_match = []
temp = df["Business Homepage"]
privacy_policy_links = df["Privacy Policy Link"]
i = 0
for t in temp:
    t2 = get_privacy_policy_link_mobile(t)
    mobile_policy_links.append(t2)
    if t2 == privacy_policy_links[i]:
        mobile_policy_links_match.append(True)
    elif privacy_policy_links[i] == 'Homepage does not link to privacy page':
        mobile_policy_links_match.append('')
    else:
        mobile_policy_links_match.append(False)
    i += 1
df["Privacy Page linked on Mobile Homepage"] = mobile_policy_links
df["Mobile Privacy Link matches Website Link"] = mobile_policy_links_match
df.to_excel("Cleaned Data v5.xlsx", index=False)

In [ ]:
df = df[df["Mobile Privacy Link matches Website Link"] != '']
df["Mobile Privacy Link matches Website Link"].mean()

In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import string
def check_mentions(url, keywords):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req).read()
    soup = BeautifulSoup(html, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    ans = []
    #text = text.lower()
    #keywords = [k.lower() for k in keywords]
    print(text)
    for k in keywords:
        if k in text:
            indeces = [m.start() for m in re.finditer(k, text)]
            for i in indeces:
                prev = text[i - 1]
                next = text[i + len(k)]
                if prev != '\n' and next != '\n' and not prev.isupper() and not next.isupper():
                    while prev != '\n':
                        
                        prev -= 1
                    ans.append(k)
                    break
    return ans
check_mentions('https://www.artofsaving.com/privacy-policy', ["GDPR", "Colorado", "Alabama", "Alaska", "American Samoa", "Arizona", "Arkansas", "Connecticut", "Delaware", "District of Columbia", "Florida", "Georgia", "Guam", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Minor Outlying Islands", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Northern Mariana Islands", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "U.S. Virgin Islands", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"])

In [ ]:
text = """
"""
states = ['Gdpr', 'General data protection regulation', 'Colorado', "Alabama", "Alaska", "American Samoa", "Arizona", "Arkansas", "Connecticut", "Delaware", "District of Columbia", "Florida", "Georgia", "Guam", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Minor Outlying Islands", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Northern Mariana Islands", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "U.S. Virgin Islands", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
for state in states:
    if state in text:
        print(state)

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from urllib.parse import urlparse, urlunparse
from webdriver_manager.chrome import ChromeDriverManager
from boilerpy3 import extractors
from urllib.request import Request, urlopen
import re
import string
def get_visible_text(driver):
    # Get all text nodes visible on the page
    elements = driver.find_elements(By.XPATH, "//*[not(self::script) and not(self::style) and text()]")
    visible_texts = []
    for element in elements:
        if element.is_displayed():
            visible_texts.append(element.text)
    return ' '.join(visible_texts)

def check_mentions(driver, url, keywords):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, 'html'))
        )
        
        # Get the visible text from the page
        visible_text = get_visible_text(driver)
        
        # Check if any of the keywords are in the visible text
        for keyword in keywords:
            if keyword.lower() in visible_text.lower():
                return True
        return False
    except Exception as e:
        print(f"An error occurred with {url}: {e}")
        return ''
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
#print(check_mentions(driver, "https://technowood.com/", ["cookie", "cookies"]))
df = pd.read_excel("With Mobile Links.xlsx")
t1 = df["Business Homepage"]
emp = []
for i in t1:
    emp.append(check_mentions(driver, i, ["cookie", "cookies", "your privacy policy", "share my personal information", "sell my personal information", "sale of personal information"]))
df["Shares/Sells Information"] = emp
df.to_excel("With Pop-ups 2.xlsx", index=False)